In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import linalg_ops
from tensorflow.python.ops import math_ops
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns

ImportError: No module named seaborn

In [8]:
@ops.RegisterGradient("gradient_no_unitary_adjustment")
def _test1(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan with no adjustment for subspace"""
    e = op.outputs[0]
    v = op.outputs[1]
    #dim = v.get_shape()
    with ops.control_de pendencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array_ops.diag(e)
            #v_proj = arrary.ops.slice(v, [0,0], [])
            grad_grassman = grad_v# - math_ops.batch_matmul(math_ops.batch_matmul(v, array_ops.transpose(grad_v)), v)
            grad_a = math_ops.batch_matmul(grad_grassman, math_ops.batch_matmul(E, array_ops.transpose(grad_v)))+math_ops.batch_matmul(grad_v, math_ops.batch_matmul(E, array_ops.transpose(grad_grassman)))
    return grad_a

In [7]:
@ops.RegisterGradient("grassman_with_2d")
def _test1(op, grad_e, grad_v):
    """Gradient for SelfAdjointEigV2 derived with Joan with no adjustment for subspace"""
    e = op.outputs[0]
    v = op.outputs[1]
    #dim = v.get_shape()
    with ops.control_dependencies([grad_e.op, grad_v.op]):
        if grad_v is not None:  
            E = array_ops.diag(e)
            v_proj = array_ops.slice(v, [0,0], [20,2])
            grad_grassman = grad_v - math_ops.batch_matmul(math_ops.batch_matmul(v_proj, array_ops.transpose(v_proj)), grad_v)
            grad_a = math_ops.batch_matmul(grad_grassman, math_ops.batch_matmul(E, array_ops.transpose(grad_v)))+math_ops.batch_matmul(grad_v, math_ops.batch_matmul(E, array_ops.transpose(grad_grassman)))
    return grad_a

In [2]:
def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=0.5, p_out=0.1, seed=None):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out, seed=seed)
    A = nx.adjacency_matrix(G).todense()
    
    return A
def target_subspace(adj, groupsize, communities, diag, dim_proj):
    normalizer = tf.cast(2.0*groupsize*communities, dtype=tf.float64)
    total_degree = tf.cast(tf.reduce_sum(adj), dtype=tf.float64)
    r = tf.sqrt(total_degree/normalizer)
    BH_op = (tf.square(r)-1)*tf.diag(tf.ones(shape=[communities*groupsize], dtype=tf.float64))-r*adj+diag 
    val, vec = tf.self_adjoint_eig(BH_op) #this is already normalized so no need to normalize
    subspace = tf.slice(vec, [0,0], [communities*groupsize, dim_proj])
    return r, subspace

def proj_magnitude(space, vector):
    projection_op = tf.matmul(space, tf.transpose(space))
    projection = tf.matmul(projection_op, vector)
    return tf.sqrt(tf.reduce_sum(tf.square(projection))) #tf.reduce_sum(tf.abs(projection))#


def rnd_vec_normed(communities, groupsize, seed=None):
    rnd_vec1 = tf.Variable(tf.random_normal(shape=[communities*groupsize,1], mean=0.0,stddev=1.0,
                                                    dtype=tf.float64,
                                                    seed=seed))
    return normalize_vec(rnd_vec1)


In [49]:
#Let's create a dataset
communities = 2
group_size = 10

data_easy = [np.asarray(balanced_stochastic_blockmodel(communities, group_size, p, 0.1*p)).astype(np.double) for p in np.linspace(0.2, 0.8,70)]
data_hard = [np.asarray(balanced_stochastic_blockmodel(communities, group_size, p, 0.5*p)).astype(np.double) for q in np.linspace(0.2, 0.8,7)]

data = data_easy+data_hard


In [50]:
#we need to do online training for now as I cannot write eigendecomposition in batch

data[0]

array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  

In [51]:
def learn_average_deg_variable(communities = 2, group_size = 10, seed=1, seed_v=None, name='gradient_no_unitary_adjustment', projection_dim=2, 
                     print_ratio=10, l_rate=0.1, mean=0.3, sd=1.0):
    """testing to see if the loss will decrease backproping through very simple function"""
    
    #now p and q will be generated from a range of 
    
    X = tf.placeholder(dtype=tf.float64, shape=[communities*group_size, communities*group_size])
    
    B = tf.cast(X, dtype = tf.float64)
    
    Diag = tf.diag(tf.reduce_sum(B,0))
    Diag = tf.cast(Diag, tf.float64)
    
    #by symmetry I should make this a bit more constrained.  so

    v =  tf.Variable(tf.random_normal(shape=[communities*group_size,1], mean=mean,
                                 stddev=sd, dtype=tf.float64,
                                 seed=seed_v, name=None))
    
     
    
    degree = tf.cast(communities*group_size, dtype=tf.float64)
    r_param = tf.div(tf.cast(1.0, dtype=tf.float64), degree)*tf.matmul(tf.transpose(v), tf.matmul(Diag, v))

    
    BH = (tf.square(r_param)-1)*tf.diag(tf.ones(shape=[communities*group_size], dtype=tf.float64))-tf.mul(r_param, B)+Diag 
    

    with tf.Session() as sess:
        g = tf.get_default_graph()
        
        with g.gradient_override_map({'SelfAdjointEigV2': name}):
            eigenval, eigenvec = tf.self_adjoint_eig(BH)
            #we try to do svm in this subspace 
            #or we can project it down to 1 dimensions, do the clustering there via some threshold and check if it makes sense 
            #by computing the loss, if it is too big, we change the angle we project down to...
            
            
            eigenvec_proj = tf.slice(eigenvec, [0,0], [communities*group_size, projection_dim])
            
            
            
            true_assignment_a = tf.concat(0, [-1*tf.ones([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
            true_assignment_b = -1*true_assignment_a
            true_assignment_a = tf.expand_dims(true_assignment_a, 1)
            true_assignment_b = tf.expand_dims(true_assignment_b, 1)

            
            projected_a = tf.matmul(tf.matmul(eigenvec_proj, tf.transpose(eigenvec_proj)), true_assignment_a)#tf.transpose(true_assignment_a))
            projected_b = tf.matmul(tf.matmul(eigenvec_proj, tf.transpose(eigenvec_proj)), true_assignment_b)#tf.transpose(true_assignment_b))
            
            
            
            loss = tf.minimum(tf.reduce_sum(tf.square(tf.sub(projected_a, true_assignment_a))),
                              tf.reduce_sum(tf.square(tf.sub(projected_b, true_assignment_b))))
            
            optimizer = tf.train.GradientDescentOptimizer(l_rate)
            
            train = optimizer.minimize(loss, var_list=[v])

            eigenvec_grad = tf.gradients(eigenvec, v)
            loss_grad = tf.gradients(loss, v)
            
            
            
            r_op, target = target_subspace(adj=B, groupsize=group_size, communities=communities, diag=Diag, dim_proj=projection_dim)  
            
            r_op_projection_a = tf.matmul(tf.matmul(target, tf.transpose(target)), true_assignment_a)
            r_op_projection_b = tf.matmul(tf.matmul(target, tf.transpose(target)), true_assignment_b)
            r_op_loss = tf.minimum(tf.reduce_sum(tf.square(tf.sub(r_op_projection_a, true_assignment_a))),
                              tf.reduce_sum(tf.square(tf.sub(r_op_projection_b, true_assignment_b))))
            
            init = tf.initialize_all_variables()
            
            
            sess.run(init)
            a,r, b,c,d= sess.run([v, r_param, r_op_loss, r_op, tf.transpose(r_op_projection_a)], feed_dict={X:data[0]})
            a_lst = []
            r_lst = []
            b_lst = []
            c_lst = []
            d_lst = []
            
            a_lst.append(a)
            r_lst.append(r)
            b_lst.append(b)
            c_lst.append(None)
            d_lst.append(d)
            
            print "initial v: {}. r_param: {}. r_op = sqrt(average degree) : {} . Loss associated with r_op: {}. r_op assignments {}.".format(a, r, c, b, d)
            for i in range(len(data)):   
                try:
                    sess.run(feed_dict={X:data[i]})
                    sess.run(train)
                except: 
                    pass
                
                if i%print_ratio==0:  
                    #print i
                    try:
                        a,r, b,c,d = sess.run([v, r_param, loss, tf.gradients(loss, v), tf.transpose(projected_a)], feed_dict={X:data[i]}) 
                        a_lst.append(a)
                        r_lst.append(r)
                        b_lst.append(b)
                        c_lst.append(c)
                        d_lst.append(d)
                    except:
                        a,r, b,c,d = None, None, None, None, None 
                        a_lst.append(a)
                        r_lst.append(r)
                        b_lst.append(b)
                        c_lst.append(c)
                        d_lst.append(d)
                    #print "current r: {}, current loss: {}, gradient of loss/r is {} and current assignments (up to sign) {}.".format(a,b,c,d)  

    d = {"v": a_lst, "r_param": r_lst, "loss": b_lst, "gradient_loss_v": c_lst, "projection": d_lst}
    d = pd.DataFrame(d)
    d.to_csv("/Users/xiangli/Desktop/clusternet/Learning_r_matrix_data/r{}rate{}p{}q{}step{}.csv".format(mean, l_rate, p, q, print_ratio))
    return  d
                

                
                

In [52]:
learn_average_deg_variable(print_ratio=1, name='grassman_with_2d',  l_rate=0.00001, mean=0.01)

initial v: [[-1.11051645]
 [-0.61363311]
 [ 2.83052531]
 [ 1.38282713]
 [-1.06472567]
 [-0.6895324 ]
 [-0.97226201]
 [ 1.96329165]
 [-2.01884377]
 [ 0.30739055]
 [-1.86144375]
 [ 0.1428617 ]
 [ 0.33484053]
 [-0.03850554]
 [-0.21483851]
 [-0.31516606]
 [-0.247081  ]
 [-1.18839329]
 [ 2.2798943 ]
 [-0.28851202]]. r_param: [[ 2.85293892]]. r_op = sqrt(average degree) : 1.0246950766 . Loss associated with r_op: 13.669704119. r_op assignments [[ -1.00936041e+00  -9.85498804e-01   2.89462827e-01   2.98519675e-01
   -1.00947708e+00   3.01908777e-01  -9.85498804e-01   3.02961922e-01
    2.99139040e-01  -1.00947708e+00   3.16293343e-01   3.16657124e-01
   -3.73614824e-15   3.11624999e-01   3.06192920e-01   3.08549218e-01
    3.17608657e-01   3.19297207e-01   3.08549218e-01   3.18203251e-01]].


,gradient_loss_v,loss,projection,r_param,v
0,None,13.669704,"[[-1.0093604098, -0.98549880405, 0.28946282666...",[[2.85293891862]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
1,"[[[-2012.26287237], [-555.953547393], [2564.46...",8.213144,"[[0.0, 3.43629889473e-16, -0.586287022218, -1....",[[2.85293891862]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
2,"[[[-28.6529282424], [0.0], [73.0316406088], [3...",11.666371,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.084405708072...",[[1.3331336587]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
3,"[[[-17050.370584], [-14132.1705141], [86917.22...",4.338170,"[[-0.156594063823, -0.987131959899, -0.6895576...",[[5.10417795762]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
4,"[[[-17617.4962844], [-9734.82112102], [74840.2...",4.806160,"[[-0.963644028344, -0.767615417017, -1.2542447...",[[4.6105110321]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
5,"[[[-1486.04275662], [-821.135999141], [7575.36...",8.261024,"[[-0.566484005633, -0.444783261295, -0.2893754...",[[3.08427005148]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
6,"[[[-2534.02859294], [-700.108422604], [3229.41...",4.991255,"[[-0.876975395907, -0.667173965861, 0.34543655...",[[2.78501377992]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
7,"[[[-2671.89175172], [-4429.18582263], [13620.4...",9.200165,"[[0.387704876182, -1.21393887118, -1.075356469...",[[3.45614319015]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
8,"[[[-60474.2205613], [-22277.3130413], [102759....",6.343155,"[[-1.10477203349, 0.101392454216, -1.080886567...",[[5.24015580354]],"[[-1.11051645499], [-0.61363311033], [2.830525..."
9,"[[[-4012.69334793], [-6651.82804497], [20455.4...",15.197822,"[[-0.205844914041, -0.667461544041, -0.1629767...",[[3.98632671793]],"[[-1.11051645499], [-0.61363311033], [2.830525..."


In [4]:
def learn_average_deg(communities = 2, group_size = 10, seed=1, seed_v=None, p=0.8, q=0.05, name='gradient_no_unitary_adjustment', projection_dim=2, iterations=100, 
                     print_ratio=10, l_rate=0.1, mean=0.3, sd=1.0):
    """testing to see if the loss will decrease backproping through very simple function"""
    B = np.asarray(balanced_stochastic_blockmodel(communities, group_size, p, q, seed)).astype(np.double)
    B = tf.cast(B, dtype = tf.float64)
    
    Diag = tf.diag(tf.reduce_sum(B,0))
    Diag = tf.cast(Diag, tf.float64)
    
    #by symmetry I should make this a bit more constrained.  so

    v_init =  tf.Variable(tf.random_normal(shape=[1], mean=mean,
                                 stddev=sd, dtype=tf.float64,
                                 seed=seed_v, name=None))
    v=v_init*tf.ones([communities*group_size, 1], dtype=tf.float64)
    
     
    
    degree = tf.cast(communities*group_size, dtype=tf.float64)
    r_param = tf.div(tf.cast(1.0, dtype=tf.float64), degree)*tf.matmul(tf.transpose(v), tf.matmul(Diag, v))

    
    BH = (tf.square(r_param)-1)*tf.diag(tf.ones(shape=[communities*group_size], dtype=tf.float64))-tf.mul(r_param, B)+Diag 
    

    with tf.Session() as sess:
        g = tf.get_default_graph()
        
        with g.gradient_override_map({'SelfAdjointEigV2': name}):
            eigenval, eigenvec = tf.self_adjoint_eig(BH)
            #we try to do svm in this subspace 
            #or we can project it down to 1 dimensions, do the clustering there via some threshold and check if it makes sense 
            #by computing the loss, if it is too big, we change the angle we project down to...
            
            
            eigenvec_proj = tf.slice(eigenvec, [0,0], [communities*group_size, projection_dim])
            
            
            
            true_assignment_a = tf.concat(0, [-1*tf.ones([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
            true_assignment_b = -1*true_assignment_a
            true_assignment_a = tf.expand_dims(true_assignment_a, 1)
            true_assignment_b = tf.expand_dims(true_assignment_b, 1)

            
            projected_a = tf.matmul(tf.matmul(eigenvec_proj, tf.transpose(eigenvec_proj)), true_assignment_a)#tf.transpose(true_assignment_a))
            projected_b = tf.matmul(tf.matmul(eigenvec_proj, tf.transpose(eigenvec_proj)), true_assignment_b)#tf.transpose(true_assignment_b))
            
            
            
            loss = tf.minimum(tf.reduce_sum(tf.square(tf.sub(projected_a, true_assignment_a))),
                              tf.reduce_sum(tf.square(tf.sub(projected_b, true_assignment_b))))
            
            optimizer = tf.train.GradientDescentOptimizer(l_rate)
            
            train = optimizer.minimize(loss, var_list=[v_init])

            eigenvec_grad = tf.gradients(eigenvec, v)
            loss_grad = tf.gradients(loss, v)
            
            
            
            r_op, target = target_subspace(adj=B, groupsize=group_size, communities=communities, diag=Diag, dim_proj=projection_dim)  
            
            r_op_projection_a = tf.matmul(tf.matmul(target, tf.transpose(target)), true_assignment_a)
            r_op_projection_b = tf.matmul(tf.matmul(target, tf.transpose(target)), true_assignment_b)
            r_op_loss = tf.minimum(tf.reduce_sum(tf.square(tf.sub(r_op_projection_a, true_assignment_a))),
                              tf.reduce_sum(tf.square(tf.sub(r_op_projection_b, true_assignment_b))))
            
            init = tf.initialize_all_variables()
            
            
            sess.run(init)
            a,r, b,c,d= sess.run([v, r_param, r_op_loss, r_op, tf.transpose(r_op_projection_a)])
            a_lst = []
            r_lst = []
            b_lst = []
            c_lst = []
            d_lst = []
            
            a_lst.append(a)
            r_lst.append(r)
            b_lst.append(b)
            c_lst.append(None)
            d_lst.append(d)
            
            print "initial v: {}. r_param: {}. r_op = sqrt(average degree) : {} . Loss associated with r_op: {}. r_op assignments {}.".format(a, r, c, b, d)
            for i in range(iterations):   
                try: sess.run(train)
                except: 
                    pass
                
                if i%print_ratio==0:  
                    #print i
                    try:
                        a,r, b,c,d = sess.run([v, r_param, loss, tf.gradients(loss, v), tf.transpose(projected_a)]) 
                        a_lst.append(a)
                        r_lst.append(r)
                        b_lst.append(b)
                        c_lst.append(c)
                        d_lst.append(d)
                    except:
                        a,r, b,c,d = None, None, None, None, None 
                        a_lst.append(a)
                        r_lst.append(r)
                        b_lst.append(b)
                        c_lst.append(c)
                        d_lst.append(d)
                    #print "current r: {}, current loss: {}, gradient of loss/r is {} and current assignments (up to sign) {}.".format(a,b,c,d)  

    d = {"v": a_lst, "r_param": r_lst, "loss": b_lst, "gradient_loss_v": c_lst, "projection": d_lst}
    d = pd.DataFrame(d)
    d.to_csv("/Users/xiangli/Desktop/clusternet/Learning_r_matrix_data/r{}rate{}p{}q{}iterations{}step{}.csv".format(mean, l_rate, p, q, iterations, print_ratio))
    return  d
                

                
                

In [10]:
learn_average_deg(print_ratio=10, iterations=1000, name='grassman_with_2d',  l_rate=0.000001, mean=0.02, sd=0.01)

initial v: [[ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]
 [ 0.00142604]]. r_param: [[  1.56587021e-05]]. r_op = sqrt(average degree) : 1.96214168703 . Loss associated with r_op: 0.239352945504. r_op assignments [[-0.81008947 -0.89831384 -1.06456991 -1.0936426  -1.05592124 -0.92168635
  -0.88318609 -1.01724697 -1.0936426  -1.05086035  0.98362854  0.98397103
   1.03054178  0.80692791  1.13586066  0.80914196  1.03176237  1.10198787
   1.13586066  0.85180488]].


,gradient_loss_v,loss,projection,r_param,v
0,None,0.239353,"[[-0.810089472142, -0.89831383716, -1.06456991...",[[1.56587021022e-05]],"[[0.00142604266262], [0.00142604266262], [0.00..."
1,"[[[-21.9455510375], [-24.6887449172], [-21.945...",16.085312,"[[-1.20214030207e-05, -3.17229250731e-10, -1.2...",[[2.40409873533e-05]],"[[0.00176697655609], [0.00176697655609], [0.00..."
2,"[[[-188.258740313], [-211.791082852], [-188.25...",16.050976,"[[-0.000882128253144, -1.70158531164e-06, -0.0...",[[0.00175451619827]],"[[0.0150950073378], [0.0150950073378], [0.0150..."
3,"[[[-1900.75334565], [-2138.34751385], [-1900.7...",11.057739,"[[-0.157068080165, -0.0424379045434, -0.182834...",[[0.15615139669]],"[[0.142405765921], [0.142405765921], [0.142405..."
4,"[[[-22.6663460208], [-25.4996392734], [-22.666...",2.524406,"[[-0.691895992238, -0.525499902267, -0.8709095...",[[0.361330768192]],"[[0.216624268703], [0.216624268703], [0.216624..."
5,"[[[-2.90896716855], [-3.27258806461], [-2.9089...",2.396388,"[[-0.700278292548, -0.537719418282, -0.8825839...",[[0.368054892098]],"[[0.218630594008], [0.218630594008], [0.218630..."
6,"[[[-0.424166957854], [-0.477187827586], [-0.42...",2.379583,"[[-0.701373387876, -0.539340873451, -0.8841139...",[[0.368972472625]],"[[0.218902953243], [0.218902953243], [0.218902..."
7,"[[[-0.062928319158], [-0.0707943590527], [-0.0...",2.377126,"[[-0.70153343375, -0.53957834987, -0.884337639...",[[0.369107378398]],"[[0.218942967845], [0.218942967845], [0.218942..."
8,"[[[-0.00935962762062], [-0.0105295810732], [-0...",2.376761,"[[-0.70155718567, -0.539613604036, -0.88437084...",[[0.369127416952]],"[[0.218948910886], [0.218948910886], [0.218948..."
9,"[[[-0.0013926270873], [-0.00156670547321], [-0...",2.376707,"[[-0.70156071858, -0.539618848059, -0.88437577...",[[0.369130397918]],"[[0.218949794968], [0.218949794968], [0.218949..."


In [11]:
learn_average_deg(print_ratio=10, iterations=1000, name='grassman_with_2d',  l_rate=0.000001, mean=0.02, sd=0.01)

initial v: [[ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]
 [ 0.03108299]]. r_param: [[ 0.00743937]]. r_op = sqrt(average degree) : 1.96214168703 . Loss associated with r_op: 0.239352945504. r_op assignments [[-0.81008947 -0.89831384 -1.06456991 -1.0936426  -1.05592124 -0.92168635
  -0.88318609 -1.01724697 -1.0936426  -1.05086035  0.98362854  0.98397103
   1.03054178  0.80692791  1.13586066  0.80914196  1.03176237  1.10198787
   1.13586066  0.85180488]].


,gradient_loss_v,loss,projection,r_param,v
0,None,0.239353,"[[-0.810089472142, -0.89831383716, -1.06456991...",[[0.00743937324168]],"[[0.0310829916361], [0.0310829916361], [0.0310..."
1,"[[[-493.144643525], [-554.787723966], [-493.14...",15.849644,"[[-0.00596620368654, -7.62234133467e-05, -0.00...",[[0.0115003688917]],"[[0.0386465315588], [0.0386465315588], [0.0386..."
2,"[[[-105.748669957], [-118.967253702], [-105.74...",2.988449,"[[-0.661005030104, -0.482996985464, -0.8283781...",[[0.340200592413]],"[[0.210194897937], [0.210194897937], [0.210194..."
3,"[[[-9.0225096461], [-10.1503233519], [-9.02250...",2.437021,"[[-0.697625210768, -0.53381568565, -0.87888198...",[[0.365870678554]],"[[0.217980899483], [0.217980899483], [0.217980..."
4,"[[[-1.26238101186], [-1.42017863835], [-1.2623...",2.385272,"[[-0.701002848785, -0.538791561654, -0.8835961...",[[0.368660927849]],"[[0.218810517521], [0.218810517521], [0.218810..."
5,"[[[-0.186187720314], [-0.209461185353], [-0.18...",2.377965,"[[-0.701478799652, -0.539497269149, -0.8842612...",[[0.369061303051]],"[[0.21892930219], [0.21892930219], [0.21892930..."
6,"[[[-0.0276685719185], [-0.0311271434083], [-0....",2.376886,"[[-0.701549067096, -0.539601553594, -0.8843594...",[[0.369120567121]],"[[0.218946879379], [0.218946879379], [0.218946..."
7,"[[[-0.00411629993374], [-0.00463083742546], [-...",2.376725,"[[-0.701559510775, -0.539617055264, -0.8843740...",[[0.369129378796]],"[[0.218949492722], [0.218949492722], [0.218949..."
8,"[[[-0.000612490454572], [-0.000689051761393], ...",2.376702,"[[-0.701561064532, -0.53961936157, -0.88437626...",[[0.369130689827]],"[[0.218949881541], [0.218949881541], [0.218949..."
9,"[[[-9.11385976603e-05], [-0.000102530922368], ...",2.376698,"[[-0.701561295726, -0.539619704742, -0.8843765...",[[0.369130884906]],"[[0.218949939397], [0.218949939397], [0.218949..."


In [12]:
learn_average_deg(print_ratio=10, iterations=1000, name='grassman_with_2d',  l_rate=0.0000001, mean=0.02, sd=0.01)

initial v: [[ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]
 [ 0.02827777]]. r_param: [[ 0.00615717]]. r_op = sqrt(average degree) : 1.96214168703 . Loss associated with r_op: 0.239352945504. r_op assignments [[-0.81008947 -0.89831384 -1.06456991 -1.0936426  -1.05592124 -0.92168635
  -0.88318609 -1.01724697 -1.0936426  -1.05086035  0.98362854  0.98397103
   1.03054178  0.80692791  1.13586066  0.80914196  1.03176237  1.10198787
   1.13586066  0.85180488]].


,gradient_loss_v,loss,projection,r_param,v
0,None,0.239353,"[[-0.810089472142, -0.89831383716, -1.06456991...",[[0.0061571680183]],"[[0.0282777688329], [0.0282777688329], [0.0282..."
1,"[[[-365.28212128], [-410.94238644], [-365.2821...",15.955482,"[[-0.00329679688139, -2.35240805969e-05, -0.00...",[[0.0064595403521]],"[[0.0289637936816], [0.0289637936816], [0.0289..."
2,"[[[-469.078244531], [-527.713025098], [-469.07...",15.871886,"[[-0.00540567933653, -6.27106357627e-05, -0.00...",[[0.0104554987724]],"[[0.0368491110264], [0.0368491110264], [0.0368..."
3,"[[[-607.435326652], [-683.364742483], [-607.43...",15.729510,"[[-0.00899081165393, -0.000171156623574, -0.00...",[[0.0170178282239]],"[[0.0470117810931], [0.0470117810931], [0.0470..."
4,"[[[-797.033300431], [-896.662462985], [-797.03...",15.477845,"[[-0.0153250875172, -0.000486939982278, -0.015...",[[0.0279493645665]],"[[0.0602477186791], [0.0602477186791], [0.0602..."
5,"[[[-1066.07725928], [-1199.3369167], [-1066.07...",15.005730,"[[-0.0273008885289, -0.00149729625461, -0.0285...",[[0.0465651186409]],"[[0.0777651464688], [0.0777651464688], [0.0777..."
6,"[[[-1451.66821413], [-1633.12674089], [-1451.6...",14.039003,"[[-0.0529615409178, -0.00539419871248, -0.0573...",[[0.0792078931187]],"[[0.101423564156], [0.101423564156], [0.101423..."
7,"[[[-1868.74290953], [-2102.33577322], [-1868.7...",11.901952,"[[-0.122315719365, -0.0267543721794, -0.140228...",[[0.136720582677]],"[[0.133251340812], [0.133251340812], [0.133251..."
8,"[[[-1487.61627516], [-1673.56830956], [-1487.6...",8.023011,"[[-0.320624290098, -0.145906357612, -0.3861384...",[[0.217575800598]],"[[0.168096988314], [0.168096988314], [0.168096..."
9,"[[[-676.369161829], [-760.915307058], [-676.36...",5.203281,"[[-0.508419118136, -0.309525560726, -0.6256974...",[[0.274516337378]],"[[0.188815974925], [0.188815974925], [0.188815..."


In [17]:
?np.linspace

In [55]:
l_rate_lst = [10**(-i)/3 for i in range(4, 10, 1)]

In [56]:
l_rate_lst

[3.3333333333333335e-05, 3.3333333333333337e-06]